In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from tqdm import tqdm   #for文の進捗を確認

In [2]:
load_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=040&bs=040&ta=20&sc=20214&sngz=&po1=25&pc=50"

res = requests.get(load_url)
res.encoding = "utf-8"
soup = BeautifulSoup(res.text, "html.parser")

In [3]:
# ページ数の取得
pages = soup.find("ol", class_="pagination-parts")
num_of_pages = int(pages.find_all("li")[-1].text)

In [4]:
# データフレーム作成
rental_property_datas = pd.DataFrame(columns=["名前", "住所", "最寄り駅１","最寄り駅２","最寄り駅３", "築年数", "建物全体の階数", "階数", "賃料", "管理費", "敷金", "礼金", "間取り", "面積" ])

In [5]:
for p in tqdm(range(num_of_pages)):

    # topのページは"&page=1"を付けても開ける
    page_url = load_url + "&page=" + str(p+1)
    res = requests.get(page_url)
    res.encoding = "utf-8"
    soup = BeautifulSoup(res.text, "html.parser")
    
    # サーバー負荷を避けるため1s遅延
    time.sleep(1)

    # 「建物ごとに表示」ページから物件毎の情報を取得
    cassetitems = soup.find_all("div", class_="cassetteitem")

    for i in range(len(cassetitems)):
        
        # 上の部分（建物情報）
        details = cassetitems[i].find_all("div", class_="cassetteitem-detail")
        
        for ii in range(len(details)):
            name = details[ii].find("div", class_="cassetteitem_content-title").text
            address = details[ii].find("li", class_="cassetteitem_detail-col1").text
            _stations = details[ii].find_all("div", class_="cassetteitem_detail-text")
            station1 = _stations[0].text
            station2 = _stations[1].text
            station3 = _stations[2].text
            _col3 = details[ii].find_all("li", class_="cassetteitem_detail-col3")
            building_age = _col3[0].find_all("div")[0].text
            number_of_floors = _col3[0].find_all("div")[1].text
            
            # 下の部分（部屋の情報）
            items = cassetitems[i].find("div", class_="cassetteitem-item")
            tbodys = items.find_all("tbody")
            for iii in range(len(tbodys)):
                _tds = tbodys[iii].find_all("td")
                floor = _tds[2].text.replace('\r','').replace('\n','').replace('\t','')
                rent = _tds[3].find("span", class_="cassetteitem_other-emphasis ui-text--bold").text
                maintenance_fee = _tds[3].find("span", class_="cassetteitem_price cassetteitem_price--administration").text
                deposit = _tds[4].find("span", class_="cassetteitem_price cassetteitem_price--deposit").text
                gratuity = _tds[4].find("span", class_="cassetteitem_price cassetteitem_price--gratuity").text
                layout = _tds[5].find("span", class_="cassetteitem_madori").text
                area = _tds[5].find("span", class_="cassetteitem_menseki").text
                
                # DataFrameにまとめた後、rental_property_datasへ追加する
                _d = pd.DataFrame()
                _d["名前"] = [name]
                _d["住所"] = [address]
                _d["最寄り駅１"] = [station1]
                _d["最寄り駅２"] = [station2]
                _d["最寄り駅３"] = [station3]
                _d["築年数"] = [building_age]
                _d["建物全体の階数"] = [number_of_floors]
                _d["階数"] = [floor]
                _d["賃料"] = [rent]
                _d["管理費"] = [maintenance_fee]
                _d["敷金"] = [deposit]
                _d["礼金"] = [gratuity]
                _d["間取り"] = [layout]
                _d["面積"] = [area]

                rental_property_datas = pd.concat([rental_property_datas, _d], ignore_index=True)

                

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:11<00:00,  1.97s/it]


In [6]:
rental_property_datas.head()

,名前,住所,最寄り駅１,最寄り駅２,最寄り駅３,築年数,建物全体の階数,階数,賃料,管理費,敷金,礼金,間取り,面積
0,ヒルサイドケンタッキー,長野県茅野市ちの横内,ＪＲ中央線/茅野駅 歩13分,ＪＲ中央線/上諏訪駅 歩81分,,築25年,2階建,1階,5.3万円,4300円,-,-,2DK,44.34m2
1,リバーサイドスクウェアＡ,長野県茅野市豊平,ＪＲ中央線/茅野駅 歩54分,,,築23年,2階建,1階,4.4万円,5300円,-,-,2DK,40.04m2
2,エスポワール２１Ｃ,長野県茅野市塚原１,ＪＲ中央線/茅野駅 歩9分,,,築24年,2階建,1階,5.35万円,4700円,-,-,2DK,44.34m2
3,ＪＲ中央線 茅野駅 2階建 築15年,長野県茅野市宮川,ＪＲ中央線/茅野駅 歩17分,,,築15年,2階建,1階,5.8万円,5100円,-,8.7万円,1LDK,42.63m2
4,ＪＲ中央線 茅野駅 2階建 築15年,長野県茅野市宮川,ＪＲ中央線/茅野駅 歩17分,,,築15年,2階建,2階,6.5万円,5100円,-,9.75万円,2LDK,54.67m2


In [7]:
# 重複判定

df = rental_property_datas
df_Final = df[df[["名前","住所","賃料","管理費","間取り","階数","面積","敷金","礼金"]].duplicated()]
df_Final

,名前,住所,最寄り駅１,最寄り駅２,最寄り駅３,築年数,建物全体の階数,階数,賃料,管理費,敷金,礼金,間取り,面積
10,ノースサイドＫ,長野県茅野市豊平,ＪＲ中央線/茅野駅 バス12分 (バス停)鋳物師屋口 歩7分,,,築25年,2階建,1階,4.45万円,5300円,-,-,2DK,40.04m2
14,ＪＲ中央線 茅野駅 2階建 新築,長野県茅野市本町西,ＪＲ中央線/茅野駅 歩18分,,,新築,2階建,1階,7.35万円,4400円,-,11.03万円,1LDK,41.15m2
18,ＪＲ中央線 茅野駅 2階建 新築,長野県茅野市本町西,ＪＲ中央線/茅野駅 歩19分,,,新築,2階建,1階,7.3万円,4400円,-,10.95万円,1LDK,41.15m2
20,ＪＲ中央線 茅野駅 2階建 新築,長野県茅野市本町西,ＪＲ中央線/茅野駅 歩19分,,,新築,2階建,1階,7.5万円,4400円,-,11.25万円,1LDK,41.15m2
22,ＪＲ中央線 茅野駅 2階建 新築,長野県茅野市本町西,ＪＲ中央線/茅野駅 歩19分,,,新築,2階建,2階,7.8万円,4400円,-,11.7万円,1LDK,52.57m2
24,ＪＲ中央線 茅野駅 2階建 新築,長野県茅野市本町西,ＪＲ中央線/茅野駅 歩19分,,,新築,2階建,2階,8万円,4400円,-,12万円,1LDK,52.92m2
27,イースト・カーサＡ,長野県茅野市本町東,ＪＲ中央線/茅野駅 バス9分 (バス停)中央保育園前 歩4分,,,築5年,2階建,1階,7.3万円,3900円,-,10.95万円,1LDK,45.02m2
29,ＪＲ中央線 茅野駅 2階建 新築,長野県茅野市宮川,ＪＲ中央線/茅野駅 歩27分,,,新築,2階建,1階,7.1万円,5000円,-,10.65万円,1LDK,50.14m2
33,トランスミット3,長野県茅野市本町東,ＪＲ中央線/茅野駅 歩21分,ＪＲ中央線/青柳駅 歩112分,ＪＲ中央線/上諏訪駅 歩101分,新築,2階建,1階,7.3万円,4400円,-,10.95万円,1LDK,41.15m2
35,トランスミット3,長野県茅野市本町東,ＪＲ中央線/茅野駅 歩21分,ＪＲ中央線/青柳駅 歩112分,ＪＲ中央線/上諏訪駅 歩101分,新築,2階建,1階,7.5万円,4400円,-,11.25万円,1LDK,41.15m2


In [8]:
# csvへ保存
df_Final.to_csv('SUUMO_nagano_chino.csv', index=False, encoding='utf-8-sig')

In [9]:
# 重複判定（参考）
# suumoサイトだけから抽出している分には、そのまま比較すればよいはず
df = rental_property_datas

# 全項目で判定
print(len(df[df[["名前", "住所", "最寄り駅１","最寄り駅２","最寄り駅３", "築年数", "建物全体の階数", "階数", "賃料", "管理費", "敷金", "礼金", "間取り", "面積"]].duplicated()]))

# 建物の情報は同じ場所から抜き出すことになるので、名前と住所を確認すれば良さそう
# お金に関わる項目はすべて入れる必要がある
# 階数は当然として、間取りと面積も、同じ建物内の場所で違う可能性あるので入れないといけない
print(len(df[df[["名前","住所","賃料","管理費","間取り","階数","面積","敷金","礼金"]].duplicated()]))
print(len(df[df[["名前","住所","賃料","管理費","間取り","階数","面積"]].duplicated()]))
print(len(df[df[["名前","住所","賃料","管理費","間取り","階数"       ]].duplicated()]))
print(len(df[df[["名前","住所","賃料",        "間取り","階数","面積"]].duplicated()]))
print(len(df[df[["名前",       "賃料","管理費","間取り","階数","面積"]].duplicated()]))


54
54
56
60
59
58
